In this tutorial notebook we'll learn how to:
* Plot the `x` and `y` 1d vectors using a scatter plot
* Add scatter plot 'enable_move' interaction which lets the user move the points (there by updating the `x` and `y` attributes of the scatter plot)
* Add an additional line mark (regression line) to the same figure as the scatter plot
* Link the regression line to the updates of the scatter plot using the `observe` method on the traits of scatter plot
* Add a reset button which resets the scatter points to their original values
* Add a HTMLMath label which displays the regression line equation in real time

Fun things to try:<br>
See the impact of outliers on the regression line by moving a few points to become:
1. outliers in both `x` and `y`
* outliers only in `y`
* outliers only in `x` (leverage points!)

In [ ]:
import numpy as np

from ipywidgets import Button, VBox, HBox, HTMLMath
import bqplot.pyplot as plt
from bqplot import LinearScale

In [ ]:
def linreg(x, y):
    '''
    computes intercept and slope for a simple
    ols regression
    '''
    b = np.cov(x, y)[0, 1] / np.var(x)
    a = np.mean(y) - b * np.mean(x)
    return a, b

In [ ]:
x = np.linspace(-10, 10, 50)
y = 5 * x + 7 + np.random.randn(50) * 20

def update_regline(*args):
    # update the y attribute of the reg_line with 
    # the results of running the ols regression on 
    # x and y attributes of the scatter plot
    a, b = linreg(scatter.x, scatter.y)
    reg_line.y = a + b * reg_line.x
    
    # update the equation label
    equation_label.value = eqn_tmpl.format(a, b)

# Add a scatter plot and a regression line on the same figure
axes_options = {'x': {'label': 'X'},
                'y': {'label': 'Y'}}
fig = plt.figure(title='Linear Regression', animation_duration=1000)
                 
plt.scales(scales={'x': LinearScale(min=-30, max=30),
                   'y': LinearScale(min=-80, max=80)})

scatter = plt.scatter(x, y, colors=['orangered'], default_size=32, 
                      enable_move=True)
reg_line = plt.plot(np.arange(-30, 31), [], 'g', stroke_width=5,
                    axes_options=axes_options)

fig.layout.width = '800px'
fig.layout.height = '500px'

reset_button = Button(description='Reset', button_style='success')
reset_button.layout.margin = '0px 30px 0px 60px'

eqn_tmpl = 'Regression Line: ${:.2f} + {:.2f}x$'
equation_label = HTMLMath()

def reset_points(*args):
    '''
    resets the scatter's x and y points 
    to the original values
    '''
    with scatter.hold_sync():
        # hold_sync will send trait updates 
        # (x and y here) to front end in one trip
        scatter.x = x
        scatter.y = y

# on button click reset the scatter points
reset_button.on_click(lambda btn: reset_points())
# recompute reg line when new points are added
scatter.observe(update_regline, ['x', 'y'])

# compute the reg line
update_regline(None)

In [ ]:
VBox([fig, HBox([reset_button, equation_label])])